In [1]:
from langchain_groq import ChatGroq

In [2]:
llm=ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response=llm.invoke("the first person to land on the moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/90573335728923334-ai-content-lead-think-with-google?tag=ai-spotlight")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


AI Content Lead, Think with Google — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results6  jobs matchedAI Content Lead, Think with GoogleGurgaon, Haryana, IndiaAI Sales Specialist, Google CloudSydney NSW, Australia; Canberra ACT, AustraliaData Scientist, Extended Workforce SolutionsHyderabad, Telangana, IndiaSenior Software Developer AI/Machine Learning, Applied AIWaterloo, ON, Canada; Cambridge, MA, USAPractice Specialist, Generative AI, Google CloudSeoul, South KoreaSenior Software Engineer, Natural Language Processing, Google ResearchTel Aviv, Israel1‑6 of 6Follow Life at Google on More about usAbout usopen_in_newContact usopen_in_newPressopen_in_newRelated InformationInvestor relationsopen_in_newBlogopen_i

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'AI Content Lead, Think with Google',
  'experience': '6 years of experience in marketing',
  'skills': ['AI tools',
   'Content development',
   'Marketing',
   'Prompt engineering',
   'Leveraging external data',
   'Programming languages'],
  'description': 'Create content for executive marketing audiences using AI, partner with technical teams to optimize AI tools, and produce content pieces at a regular pace'},
 {'role': 'AI Sales Specialist, Google Cloud',
  'experience': 'Not specified',
  'skills': ['AI', 'Sales', 'Cloud computing'],
  'description': 'Not specified'},
 {'role': 'Data Scientist, Extended Workforce Solutions',
  'experience': 'Not specified',
  'skills': ['Data science', 'Extended workforce solutions'],
  'description': 'Not specified'},
 {'role': 'Senior Software Developer AI/Machine Learning, Applied AI',
  'experience': 'Not specified',
  'skills': ['AI', 'Machine learning', 'Software development'],
  'description': 'Not specified'},
 {'role': 'Pract

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://zelin.com/react-portfolio
1,"Angular,.NET, SQL Server",https://zelin.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://zelin.com/vue-portfolio
3,"Python, Django, MySQL",https://zelin.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://zelin.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://zelin.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://zelin.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://zelin.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://zelin.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://zelin.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [11]:
job = json_res[0]
job['skills']

['AI tools',
 'Content development',
 'Marketing',
 'Prompt engineering',
 'Leveraging external data',
 'Programming languages']

In [12]:

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://zelin.com/ml-python-portfolio'},
  {'links': 'https://zelin.com/ios-ar-portfolio'}],
 [{'links': 'https://zelin.com/wordpress-portfolio'},
  {'links': 'https://zelin.com/magento-portfolio'}],
 [{'links': 'https://zelin.com/magento-portfolio'},
  {'links': 'https://zelin.com/ml-python-portfolio'}],
 [{'links': 'https://zelin.com/devops-portfolio'},
  {'links': 'https://zelin.com/kotlin-backend-portfolio'}],
 [{'links': 'https://zelin.com/ios-portfolio'},
  {'links': 'https://zelin.com/ml-python-portfolio'}],
 [{'links': 'https://zelin.com/ml-python-portfolio'},
  {'links': 'https://zelin.com/magento-portfolio'}]]

In [13]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are ragavan, a business development executive at zelin.  zelin is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of zelin 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase zelin's portfolio: {link_list}
        Remember you are ragavan, BDE at zelin. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking AI-Powered Content Creation with Zelin

Dear Hiring Manager,

I came across the job description for the AI Content Lead role at Think with Google, and I was impressed by the opportunity to leverage AI in creating compelling content for executive marketing audiences. As a Business Development Executive at Zelin, I'd like to introduce our company's capabilities in empowering businesses like yours to harness the power of AI and automation.

With 6 years of experience in marketing being a key requirement for this role, I'd like to highlight Zelin's expertise in developing tailored solutions that cater to the unique needs of marketing teams. Our team has a proven track record of delivering scalable, optimized, and efficient solutions that drive business growth. We've worked with numerous enterprises, fostering process optimization, cost reduction, and heightened overall efficiency.

Our expertise in AI tools, content development, and programming languages makes us an idea